### RAG Pipelines - Data Ingestion to Vector DB pipeline


In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [ ]:
### Read all the pdfs inside the directory


def process_all_pdfs(pdf_directory):
    """Process all pdf files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)

    ## find all pdf files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            # add the source information to metadata
            for doc in documents:
                doc.metadata["source_file"] = pdf_file.name
                doc.metadata["file_type"] = "pdf"

            all_documents.extend(documents)
            print(f"✔️ Loaded {len(documents)} pages")
        except Exception as e:
            print(f"❌ Error: {e} pages")

    print(f"\n Total Documents Loaded: {len(all_documents)}")
    return all_documents


## process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 3 PDF files to process

Processing: attention.pdf
✔️ Loaded 1 pages

Processing: embeddings.pdf
✔️ Loaded 1 pages

Processing: objectdetection.pdf
✔️ Loaded 1 pages

 Total Documents Loaded: 3


In [17]:
all_pdf_documents

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-10-07T13:18:48+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-10-07T13:18:48+00:00', 'subject': '(unspecified)', 'title': 'Attention in Machine Learning', 'trapped': '/False', 'source': '..\\data\\pdf\\attention.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'attention.pdf', 'file_type': 'pdf'}, page_content='Attention in Machine Learning\nAttention in Machine Learning is a concept that allows models to focus on the most\nrelevant parts of the input data when making predictions. It was first introduced in\nthe field of Natural Language Processing (NLP) and has become a key idea behind\nmany modern AI models such as Transformers, BERT, and GPT.\nIn traditional neural networks, all input data is treated equally, which can make it\ndifficult for models to capture long-term dependencies. Attention solves this by\nassigning d

In [ ]:
### Text Splitting get into chunks
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """ "Split docuemnts into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""],
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"\nSplit {len(documents)} documents into {len(split_docs)} chunks")

    ## show example of chunk
    if split_docs:
        print("\nExample chunk:")
        print(f"Content: \n{split_docs[0].page_content}\n---")
        print(f"Metadata: {split_docs[0].metadata}")
    return split_docs

In [19]:
chunks = split_documents(all_pdf_documents)


Split 3 documents into 7 chunks

Example chunk:
Content: 
Attention in Machine Learning
Attention in Machine Learning is a concept that allows models to focus on the most
relevant parts of the input data when making predictions. It was first introduced in
the field of Natural Language Processing (NLP) and has become a key idea behind
many modern AI models such as Transformers, BERT, and GPT.
In traditional neural networks, all input data is treated equally, which can make it
difficult for models to capture long-term dependencies. Attention solves this by
assigning different “weights” to different parts of the input, meaning the model can
decide which words, pixels, or features are most important for the current task.
The mechanism works by computing a score for each input element based on its
relevance to the output being generated. These scores are then converted into
probabilities using a softmax function. The model uses these probabilities to focus
more on important elements while 

### Embedding and Vector Store DB


In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

e:\GENAi\Langchain_RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
class EmbeddingManager:
    """Handles document embedding generation using Sentence Transformer"""

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialzes the embedding manager

        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the sentence Tranformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(
                f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}"
            )
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts

        Args:
            texts: List of text strings to embed

        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not found")

        print(f"Generating embeddings for {len(texts) } texts...")
        embeddings = self.model.encode(texts,show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager = EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2


e:\GENAi\Langchain_RAG\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\badal\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regu

Model loaded successfully. Embedding dimension: 384


### Vector Store

In [43]:
class VectorStore:
    """Manages document embeddings in a chromaDB vector Store"""

    def __init__(self,collection_name:str = "pdf_documents",persist_directory:str = "../data/vector_store"):
        """
        Initialize the vector store 

        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store 
        """

        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChormaDB client and connection"""
        try:
            # create persistent ChromaDb client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            # Get or create Collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description":"PDF document embeddings for RAG"}
            )

            print(f"Vector Store initialzed. Colleciton: {self.collection.name}")
            print(f"Existing documents in collection: {self.collection.count()}")
        except Exception as e:
            print("Error initializing vector store: {e}")
            raise
    
    def add_documents(self,documents: List[Any], embeddings:np.ndarray):
        """
        Add documents and their embeddings to the vector store

        Args:
            docuements: List of langchain documents
            embeddings: Corresponding embeddings for the documents 
        """

        if(len(documents) != len(embeddings)):
            raise ValueError("Number of documents must match number of embeddings")

        print(f"Adding {len(documents)} documents to vector store...")

        ## prepare data for chroma db
        ids = []
        metadatas = []
        documents_text = []
        embedding_list = []

        for i, (doc, embedding) in enumerate(zip(documents,embeddings)):
            # Generate unique id
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_i"
            ids.append(doc_id)


            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index']= i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            # Document content
            documents_text.append(doc.page_content)

            # Embedding
            embedding_list.append(embedding.tolist())

        # Add Coolection
        try:
            self.collection.add(
                ids=ids,
                embeddings =embedding_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vector_store = VectorStore()

Vector Store initialzed. Colleciton: pdf_documents
Existing documents in collection: 7


In [44]:
### Conver the text to embeddings and add to vector store
texts = [doc.page_content for doc in chunks]

## Generate the embeddings
embeddings = embedding_manager.generate_embeddings(texts)

## store in the vector database
vector_store.add_documents(chunks,embeddings)

Generating embeddings for 7 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  4.98it/s]

Generated embeddings with shape: (7, 384)
Adding 7 documents to vector store...
Successfully added 7 documents to vector store
Total documents in collection: 14
